In [34]:
import gym
import numpy as np

In [35]:
env = gym.make("MountainCar-v0")
env.reset()

array([-0.4965397,  0.       ], dtype=float32)

In [36]:
done = False
while not done:
    action = 2
    new_state, reward, done, _ = env.step(action)
    print(reward, new_state)
    env.render()
env.close()

-1.0 [-0.49574244  0.00079728]
-1.0 [-0.49415383  0.0015886 ]
-1.0 [-0.4917858   0.00236805]
-1.0 [-0.48865595  0.00312981]
-1.0 [-0.48478773  0.00386822]
-1.0 [-0.48020995  0.0045778 ]
-1.0 [-0.47495663  0.0052533 ]
-1.0 [-0.46906686  0.00588978]
-1.0 [-0.46258426  0.00648261]
-1.0 [-0.45555672  0.00702755]
-1.0 [-0.44803593  0.00752077]
-1.0 [-0.44007704  0.00795889]
-1.0 [-0.43173802  0.00833901]
-1.0 [-0.42307928  0.00865875]
-1.0 [-0.41416305  0.00891622]
-1.0 [-0.40505293  0.00911012]
-1.0 [-0.3958133   0.00923965]
-1.0 [-0.3865087   0.00930456]
-1.0 [-0.37720358  0.00930513]
-1.0 [-0.36796147  0.00924212]
-1.0 [-0.35884467  0.00911679]
-1.0 [-0.34991384  0.00893084]
-1.0 [-0.34122747  0.00868635]
-1.0 [-0.3328417   0.00838579]
-1.0 [-0.32480976  0.00803193]
-1.0 [-0.31718192  0.00762783]
-1.0 [-0.31000516  0.00717677]
-1.0 [-0.30332294  0.00668221]
-1.0 [-0.29717517  0.00614779]
-1.0 [-0.29159793  0.00557722]
-1.0 [-0.2866236   0.00497434]
-1.0 [-0.2822806  0.004343 ]
-1.0 [-0.2

In [37]:
print(env.observation_space.high)
print(env.observation_space.low)
print(env.action_space.n)

[0.6  0.07]
[-1.2  -0.07]
3


In [38]:
LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 25000

SHOW_EVERY = 2000

In [39]:
# observation space
DISCRETE_OS_SIZE = [20]*len(env.observation_space.high)

In [40]:
# window size
discrete_os_win_size = (env.observation_space.high-env.observation_space.low)/DISCRETE_OS_SIZE
print(discrete_os_win_size)

[0.09  0.007]


In [41]:
q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))

In [42]:
print(q_table.shape)
print(q_table)

(20, 20, 3)
[[[-1.44452259 -1.57113429 -0.23376052]
  [-1.03388087 -1.19804861 -0.94713583]
  [-0.6118547  -1.27448199 -0.22587013]
  ...
  [-0.31575146 -1.75594941 -1.67586501]
  [-1.01983427 -0.77692744 -1.45188978]
  [-0.55397933 -1.37916452 -1.74029928]]

 [[-1.62072944 -1.92582377 -1.22823082]
  [-1.53887783 -0.12166343 -0.90996387]
  [-1.49183052 -0.24331821 -1.88244857]
  ...
  [-0.18075163 -1.96166898 -0.87064676]
  [-1.13761609 -1.74624159 -1.50802599]
  [-1.80276079 -0.29624626 -1.67010772]]

 [[-0.88058008 -1.84706097 -0.8599888 ]
  [-1.28133853 -1.04274859 -1.97302202]
  [-1.19647796 -1.67163343 -1.6753327 ]
  ...
  [-1.79969337 -1.04631167 -0.67713741]
  [-1.77510634 -1.78437265 -1.1003426 ]
  [-1.42624757 -0.78087361 -0.42741648]]

 ...

 [[-1.56913125 -0.43265798 -0.4703396 ]
  [-0.91948198 -1.95169192 -1.03960946]
  [-0.03442328 -1.47057838 -1.79151895]
  ...
  [-1.64669037 -1.55883305 -1.10033385]
  [-0.51245826 -0.30264237 -1.00340665]
  [-0.0629759  -1.24458302 -1.56

In [43]:
def get_discrete_state(state):
    discrete_state = (state- env.observation_space.low)/discrete_os_win_size
    return tuple(discrete_state.astype(np.int))

In [44]:
discrete_state = get_discrete_state(env.reset())
print(discrete_state)

(8, 10)


/tmp/ipykernel_649044/475210161.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return tuple(discrete_state.astype(np.int))


In [45]:
print(q_table[discrete_state])

[-0.63722466 -1.88069135 -0.21863251]


In [50]:
for episode in range(EPISODES):
    if episode%SHOW_EVERY == 0:
        print(episode)
        render=True
    else:
        render=False
    done = False
    while not done:
        action = np.argmax(q_table[discrete_state])
        new_state, reward, done, _ = env.step(action)
        new_discrete_state = get_discrete_state(new_state)
        if render:
            env.render()
        if not done:
            max_future_q = np.max(q_table[new_discrete_state])
            current_q = q_table[discrete_state+(action, )]
            new_q = (1-LEARNING_RATE)*current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
            q_table[discrete_state+(action, )] = new_q
        
        elif new_state[0] >= env.goal_position:
            print(f"We made it on episode {episode}")
            q_table[discrete_state + (action,)] = 0
        
        discrete_state = new_discrete_state
        
env.close()

0
2000
4000


/tmp/ipykernel_649044/475210161.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return tuple(discrete_state.astype(np.int))


6000
8000
10000
12000
14000
16000
18000
20000
22000
24000


In [51]:
env.close()